In [1]:
using ITensors
using FiniteSizeScaling
using ProgressMeter
using JLD2

In [6]:
function gaps(N; g)
    sites = siteinds("S=1/2",N)
    
    H = OpSum()
    for i in 1:N-1
        H += -1,"Sz",i,"Sz",i+1
    end
    for i in 1:N
        H += -g,"Sx",i
    end
    
    MPH = MPO(H,sites)
    ψ = randomMPS(sites,2)
    ϕ = randomMPS(sites,2)
    
    sweeps = Sweeps(30)
    setmaxdim!(sweeps, 10,10,10,20,20,40,80,100,200,200)
    setcutoff!(sweeps, 1E-8)
    setnoise!(sweeps, 1E-6)
    
    E_0, ψ_0 = dmrg(MPH, ψ, sweeps, outputlevel = 0)
    E_1, ψ_1 = dmrg(MPH, [ψ_0], ϕ, sweeps, weight = 20*g + 20, outputlevel = 0)
    ΔE = E_1 - E_0
    
    return ΔE
end

gaps (generic function with 1 method)

In [7]:
g = [0.2:0.05:0.4; 0.41:0.01:0.6; 0.65:0.05:1]
L = [60:5:80;]
ΔE_L = Vector{Vector{Float64}}(undef, length(L))
j = 0

for l in 1:length(L)
    ΔE = fill(0.0, length(g))
    @showprogress for i in 1:length(g)
        ΔE[i] = gaps(L[l], g = g[i])
    end
    ΔE_L[l] = ΔE
end

Progress: 100%|█████████████████████████████████████████| Time: 0:04:19
Progress: 100%|█████████████████████████████████████████| Time: 0:04:37
Progress: 100%|█████████████████████████████████████████| Time: 0:05:13
Progress: 100%|█████████████████████████████████████████| Time: 0:05:37
Progress: 100%|█████████████████████████████████████████| Time: 0:05:59


In [8]:
ΔE = hcat(ΔE_L...)
save_object("EnergyGaps.jld2",ΔE)

In [19]:
ν = load_object("ν.jld2")
x_scaled(X, L, v1, v2) = @. (((X-0.5)/0.5)) * L^(1/ν)
y_scaled(Y, L, v1, v2) = @. Y * L^v2
data=[[g[12:18], ΔE[12:18, l], L[l]] for l in 1:length(L)]
scaled_data, residuals, min_res, best_v1, best_v2 = fss_two_var(data=data, xs=x_scaled, ys=y_scaled, 
    v1i=0.5, v1f=0.5, n1=1, v2i=0.5, v2f=5, n2=5000, p=8, norm_y=true);
save_object("z", best_v2)

Optimal v1 value: 0.5
Optimal v2 value: 0.9878975795159032
Smallest residual: 0.00015534181809838087 
 
